In [11]:
# randomly sample a subset of a decklist to see the following
# initial hand composition
# first two draws


In [12]:
# imports
import pandas as pd
import random

import os

import numpy as np

In [13]:
# read in decklist
# WE ARE RUNNING 21 SWAMPS

decklist = pd.read_csv(os.path.join('data', 'decklist.csv'))

sub_ndxs = [0,1,2,3]

decklist.iloc[sub_ndxs,:]

,Unnamed: 0,name,mana_cost,cmc,colors,type_line,oracle_text,produced_mana
0,77,"Shizo, Death's Storehouse",NaN,0.0,[],Legendary Land,"{T}: Add {B}.\n{B}, {T}: Target legendary crea...",['B']
1,497,Diabolic Intent,{1}{B},2.0,['B'],Sorcery,"As an additional cost to cast this spell, sacr...",NaN
2,533,Misinformation,{B},1.0,['B'],Instant,Put up to three target cards from an opponent'...,NaN
3,610,Deadly Rollick,{3}{B},4.0,['B'],Instant,"If you control a commander, you may cast this ...",NaN


In [20]:
# add 

decklist['must_keeps'] = False

decklist.loc[decklist['type_line'].str.find('Land') != -1.0, 'must_keeps'] = True

decklist.to_csv(os.path.join('data', 'decklist.csv'))



In [ ]:
def gen_sampling_ndxs(deck_ndxs:list[int], no_must_keeps:int, no_basics:int) -> list[int]:
    
    return random.sample(deck_ndxs, 99-no_must_keeps-no_basics-1)

def decklist_from_ndxs(decklist:pd.DataFrame, ndxs:list[int]) -> pd.DataFrame:
    return decklist.iloc[ndxs, :]

def sample_decklist(decklist) -> list[int]:

    # drop must keeps from sampling pool

    ndxs = decklist[decklist['must_keeps'] == False].index.tolist()
    
    ndxs = gen_sampling_ndxs(
        deck_ndxs = ndxs, 
        no_must_keeps = decklist[decklist['must_keeps'] == True].shape[0],
        no_basics = 21 # we can solve for this 
    )

    # still need to repeat the basic lands to 100
    return sum(
        [decklist[decklist['must_keeps'] == True].index.tolist(), ndxs] , [] 
    )



    
# need to manually input # of basic lands

# subtract # of basic lands from 99 set it to some variable x

# randomly sample x cards from the decklist

# save index sets as a parquet from a dataframe

len(sample_decklist(decklist))

77

In [16]:
# maybe add in 'must keeps' and automatically add those to each 